<a href="https://colab.research.google.com/github/Niccolo-Rocchi/Medical_Imaging_project/blob/main/U_net_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentazione semantica su radiografie toraciche

### Previsione della maschera binaria dell'area affetta da Pneumotorace 

Pneuomothorax Imaging Challenge - https://siim.org/page/pneumothorax_challenge

## Caricamento Pacchetti

In [ ]:
%%capture
!pip install pydicom albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# For reading files
from pydicom import dcmread 
import glob
from google.colab import drive
# For dealing with data
import numpy as np
import pandas as pd
import re
import random
random.seed(123)
import os
# For plots
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
# For nets
import tensorflow as tf
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate as concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
# Data augmentation
import albumentations as A
import cv2
from albumentations.core.transforms_interface import DualTransform, to_tuple

## Collegamento Drive e Lettura file

In [ ]:
# Mount Drive data
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/pneumotorax_data

In [ ]:
# Data upload from `Data_Preprocesing` notebook
train_set = pd.read_csv("train_set.csv")
val_set = pd.read_csv("val_set.csv")

In [ ]:
# RLE to mask conversion provided by competition organizers with the dataset.
def rle2mask(rle, width, height):
    mask= np.zeros(width*height)

    if rle != ' -1':
      array = np.asarray([int(x) for x in rle.split()])
      starts = array[0::2]
      lengths = array[1::2]

      current_position = 0
      for index, start in enumerate(starts):
          current_position += start
          mask[current_position:current_position+lengths[index]] = 1
          current_position += lengths[index]

    return mask.reshape(width, height, order='F')

In [ ]:
# Data augmentation list
augment_list = [
    A.RandomResizedCrop(1024,1024,scale = (1,1), ratio=(0.8, 1.25),p=1),
    A.RandomBrightnessContrast(p=1,brightness_limit=0.4,contrast_limit=0.4), 
    A.ShiftScaleRotate(p=1,scale_limit=0.3,rotate_limit=30,shift_limit=0.1,border_mode=cv2.BORDER_CONSTANT), 
    A.GridDistortion(p=1,distort_limit=0.5), 
    A.ElasticTransform(p=1,border_mode=cv2.BORDER_CONSTANT)
]

In [ ]:
# Create input for Keras' fit function
class DataGenerator:

  # Method that yields (image, mask) tuple
  def data_generator(self, data):
    i = 0
    while True:
      while i < len(data):
        # Extract ID and its encoded pixels
        id, rle = data[["ImageId"," EncodedPixels"]].iloc[i]
        # Convert encoded pixels to mask
        mask = rle2mask(rle, 1024, 1024)
        # Read the image associate to ImageId
        try:
          dcm_file = dcmread(f"/content/gdrive/MyDrive/Med Imaging/train_whole/dicom_files/{id}.dcm")
        except:
          continue
        dcm_image = dcm_file.pixel_array
        # Rescale image
        image = dcm_image/255
        # Expand dimensions
        image = np.expand_dims(image, axis=-1)
        mask = np.expand_dims(mask, axis=-1)
        # Resize image and mask
        mask = tf.keras.layers.Resizing(256, 256, interpolation="nearest", crop_to_aspect_ratio=False)(mask)
        image = tf.keras.layers.Resizing(256, 256, interpolation="bilinear", crop_to_aspect_ratio=False)(image)
        yield (image, mask)
        i += 1

  def data_generator_augment(self, data):
    i = 0
    while True:
      while i < len(data):
        # Extract ID and its encoded pixels
        id, rle = data[["ImageId"," EncodedPixels"]].iloc[int(i)]
        # Convert encoded pixels to mask
        mask = rle2mask(rle, 1024, 1024)
        # Read the image associate to ImageId
        try:
          dcm_file = dcmread(f"/content/gdrive/MyDrive/Med Imaging/train_whole/dicom_files/{id}.dcm")
        except:
          continue
        dcm_image = dcm_file.pixel_array
        # Rescale image
        image = dcm_image/255
        if i != int(i):
          # Select randomly an augmentation method
          augment_method = list(np.random.choice(a = augment_list, replace = False, size = 1))
          transform = A.Compose(augment_method)
          if augment_method[0] != augment_list[1]:
            # Transform image and mask based on the augmentation method
            transformed = transform(image=image,mask=mask)
            image = transformed['image']
            mask = transformed['mask']
          else:
            image = transform(image=np.float32(image))['image']
        # Expand dimensions
        image = np.expand_dims(image, axis=-1)
        mask = np.expand_dims(mask, axis=-1)
        # Resize image and mask
        mask = tf.keras.layers.Resizing(256, 256, interpolation="nearest", crop_to_aspect_ratio=False)(mask)
        image = tf.keras.layers.Resizing(256, 256, interpolation="bilinear", crop_to_aspect_ratio=False)(image)
        yield (image, mask)
        i += 0.5

  # Create train generator for net input
  def train_generator(self, batch_size):
    # Create a tensorflow iterator
    tf_iterator = tf.data.Dataset.from_generator(lambda: self.data_generator(train_set), output_types=(tf.float64, tf.float64))
    # Create batch size
    tf_iterator = tf_iterator.batch(batch_size)
    # Return generator
    return tf_iterator

   # Create train generator for net input (augmented images)
  def train_generator_augmented(self, batch_size):
    # Create a tensorflow iterator
    tf_iterator = tf.data.Dataset.from_generator(lambda: self.data_generator_augment(train_set), output_types=(tf.float64, tf.float64))
    # Create batch size
    tf_iterator = tf_iterator.batch(batch_size)
    # Return generator
    return tf_iterator

  # Create validation generator for net input
  def val_generator(self, batch_size):
    # Create a tensorflow iterator
    tf_iterator = tf.data.Dataset.from_generator(lambda: self.data_generator(val_set), output_types=(tf.float64, tf.float64))
    # Create batch size
    tf_iterator = tf_iterator.batch(batch_size)
    # Return generator
    return tf_iterator

### Metrics

In [ ]:
# Intersection-Over-Union (IoU), also known as the Jaccard Index
def IoU(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return intersection / union

# True Positive Rate
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)

# Dice coefficient
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

### Loss functions

In [ ]:
# Focal loss
def focal_loss(gamma=2., alpha=.25):
	def focal_loss_fixed(y_true, y_pred):
		pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
		pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
		return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
	return focal_loss_fixed

# Dice loss
def dice_loss(y_true, y_pred, smooth=1):
    return 1 - dice_coef(y_true, y_pred)

# BinaryCrossEntropy

### Net definition and train

In [ ]:
# Metrics
metrics = [dice_coef, true_positive_rate, IoU]

# Batch and epochs
batch_size = 32
epochs = 20
assert(batch_size <= 32)

# Callbacks
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Net
def create_model(loss):

  # Model description
  inputs = Input((256, 256, 1))

  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)

  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

  u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
  u6 = concatenate()([u6, c4])
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
  c6 = Dropout(0.2) (c6)
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

  u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
  u7 = concatenate()([u7, c3])
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
  c7 = Dropout(0.2) (c7)
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

  u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
  u8 = concatenate()([u8, c2])
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
  c8 = Dropout(0.1) (c8)
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

  u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
  u9 = concatenate(axis=3)([u9, c1])
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
  c9 = Dropout(0.1) (c9)
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

  outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

  model = Model(inputs=[inputs], outputs=[outputs])

  # Model compilation
  model.compile(optimizer='adam', loss = loss, metrics = metrics)

  # Return model
  return model

In [ ]:
# Net input generator
train_generator = DataGenerator().train_generator(batch_size)
train_generator_augmented = DataGenerator().train_generator_augmented(batch_size)
val_generator = DataGenerator().val_generator(batch_size)

### Model trained with Dice Loss - No Augmentation

In [ ]:
# Loss
loss_1 = dice_loss

# Create model
dice_model = create_model(loss_1)

# Fit model
model_focal_history = dice_model.fit(x = train_generator, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

### Model trained with Dice Loss - Augmentation

In [ ]:
# Loss
loss_1 = dice_loss

# Create model
dice_model_aug = create_model(loss_1)

# Fit model
model_focal_aug_history = dice_model_aug.fit(x = train_generator_augmented, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)*2//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

### Model trained with Binary Cross Entropy Loss - No Augmentation

In [ ]:
# Loss
loss_2 = 'BinaryCrossentropy'

# Create model
bce_model = create_model(loss_2)

# Fit model
model_bce_history = bce_model.fit(x = train_generator, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

### Model trained with Binary Cross Entropy Loss - Augmentation

In [ ]:
# Loss
loss_2 = 'BinaryCrossentropy'

# Create model
bce_model_aug = create_model(loss_2)

# Fit model
model_bce_aug_history = bce_model_aug.fit(x = train_generator_augmented, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)*2//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

### Model trained with Focal Loss - No Augmentation

In [ ]:
# Loss
loss_3 = focal_loss()

# Create model
focal_model = create_model(loss_3)

# Fit model
model_focal_history = focal_model.fit(x = train_generator, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

### Model trained with Focal Loss - Augmentation

In [ ]:
# Loss
loss_3 = focal_loss()

# Create model
focal_model_aug = create_model(loss_3)

# Fit model
model_focal_aug_history = focal_model_aug.fit(x = train_generator_augmented, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)*2//batch_size),
                          validation_data = val_generator,
                          validation_steps = (len(val_set)//batch_size),
                          callbacks = callback, 
                          )

In [ ]:
# Model initialization
model_focal_aug = Model(inputs=[inputs], outputs=[outputs])
# Model compilation
model_focal_aug.compile(optimizer='adam', loss=focal_loss(), metrics=[dice_coef,true_positive_rate,IoU])

In [ ]:
# Model fit
model_history_focal_aug = model_focal_aug.fit(x = train_generator_augmented, 
                          epochs = epochs, 
                          steps_per_epoch=(len(train_set)*2//batch_size),
                          validation_data = val_generator,
                          validation_steps = 10,
                          callbacks = callback 
                          )

## Mask Prediction Example

In [ ]:
# Loss
loss_1 = dice_loss

# Create model
dice_model = create_model(loss_1)

# Restore the weights
dice_model.load_weights('/content/gdrive/MyDrive/Med Imaging/trained_models/dice_loss_model/dice_loss')

history_dice = pd.read_csv('/content/gdrive/MyDrive/Med Imaging/trained_models_history/dice_loss')
history_dice

In [ ]:
# Loss
loss_2 = 'BinaryCrossentropy'

# Create model
bce_model = create_model(loss_2)

# Restore the weights
bce_model.load_weights('/content/gdrive/MyDrive/Med Imaging/trained_models/bce_model/bce_loss')

history_bce = pd.read_csv('/content/gdrive/MyDrive/Med Imaging/trained_models_history/bce_loss')
history_bce

In [ ]:
# Loss
loss_3 = focal_loss()

# Create model
focal_model = create_model(loss_3)

# Restore the weights
focal_model.load_weights('/content/gdrive/MyDrive/Med Imaging/trained_models/focal_model/focal_loss')

history_focal = pd.read_csv('/content/gdrive/MyDrive/Med Imaging/trained_models_history/focal_loss')
history_focal

In [ ]:
for k,idx in enumerate(encoded_pixels[:32]['ImageId']): 
  dcmHead_4 = dcmread(f'/content/gdrive/MyDrive/Med Imaging/train_whole/dicom_files/{idx}.dcm') #confronto con immagine ridotta
  img_pred = dcmHead_4.pixel_array/255
  img_pred = np.expand_dims(img_pred, axis=-1)
  img_pred = tf.keras.layers.Resizing(256, 256, interpolation="bilinear", crop_to_aspect_ratio=False)(img_pred)
  test_set[k] = img_pred

test_preds = focal_model.predict(test_set)
preds_test_thresh = (test_preds <= 0.01).astype(np.uint8)